# **電商如何以A/B Test 驗證新網頁設計有效**

數據 : ab_data.csv from Kaggle

目標：在指定alpha=0.05,p1=0.12,p2=0.11,power=0.8的情況下
決定

1. reqired_n?

2. 判斷是否有效?

In [19]:
#import necesary library
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
from math import ceil

In [20]:
# Reading data
raw_data=pd.read_csv('ab_data.csv')
raw_data.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [21]:
#check unique
raw_data['user_id'].value_counts()

637561    2
821876    2
643869    2
938802    2
916765    2
         ..
710897    1
708848    1
665839    1
663790    1
630836    1
Name: user_id, Length: 290584, dtype: int64

In [22]:
# Decide the effect_size base on prob.1 and prob.2
import statsmodels.stats.api as sms
prob1=0.12
prob2=0.11
effect_size=sms.proportion_effectsize(prob1,prob2)
print('effect size=\t',effect_size)
# Decide reuqired number of sample
reuqired_n=sms.NormalIndPower().solve_power(effect_size=effect_size,power=0.8,alpha=0.05,ratio=1)
reuqired_n=ceil(reuqired_n)
print('required n=\t',reuqired_n)

effect size=	 0.031352702218681694
required n=	 15970


In [23]:
data=raw_data.drop_duplicates(subset=['user_id'],keep=False)

In [24]:
#seperate into "control" and "treatment"
control=data[data['group']=='control']
treatment=data[data['group']=='treatment']

control_sample=control.sample(reuqired_n,random_state=22)
treatment_sample=treatment.sample(reuqired_n,random_state=22)

In [31]:
ab_test_data=pd.concat([control_sample,treatment_sample])
ab_test_data=ab_test_data.reset_index(drop=True)
print(ab_test_data.head(10))

   user_id                   timestamp    group landing_page  converted
0   763854  2017-01-21 03:43:17.188315  control     old_page          0
1   690555  2017-01-18 06:38:13.079449  control     old_page          0
2   861520  2017-01-06 21:13:40.044766  control     old_page          0
3   630778  2017-01-05 16:42:36.995204  control     old_page          0
4   656634  2017-01-04 15:31:21.676130  control     old_page          0
5   846354  2017-01-12 17:34:33.014616  control     old_page          0
6   879802  2017-01-15 15:26:31.018326  control     old_page          0
7   860778  2017-01-12 00:49:32.771388  control     old_page          0
8   630816  2017-01-13 08:43:01.536244  control     old_page          0
9   889771  2017-01-03 00:55:02.444223  control     old_page          0


In [26]:
#以函數計算z_stat, pval, confidence interval

control_reuslt=ab_test_data[ab_test_data['group']=='control']['converted']
treatment_result=ab_test_data[ab_test_data['group']=='treatment']['converted']

n_control=control_reuslt.count()
n_treatment=treatment_result.count()

In [27]:
successes=[control_reuslt.sum(),treatment_result.sum()]
nobs=[n_control,n_treatment]
print(successes)
print(nobs)

[1932, 1928]
[15970, 15970]


In [28]:
from statsmodels.stats.proportion import proportion_confint, proportions_ztest
#計算z, p-value
z_stat,pval=proportions_ztest(count=successes,nobs=nobs)
#計算信賴區間
(lcl_control,lcl_treatment),(ucl_control,ucl_treatment)=proportion_confint(count=successes,nobs=nobs)

In [29]:
print("('z_stat','pval')=({0:.3f},{1:.3f})".format(z_stat,pval))
print("ci 95% for control group: (LCL,UCL)=[{0:.3f},{1:.3f}]".format(lcl_control,ucl_control))
print("ci 95% for treatment group: (LCL,UCL)=[{0:.3f},{1:.3f}]".format(lcl_treatment,ucl_treatment))
print('>>>>>>>> 效果不顯著 <<<<<<<<')

('z_stat','pval')=(0.069,0.945)
ci 95% for control group: (LCL,UCL)=[0.116,0.126]
ci 95% for treatment group: (LCL,UCL)=[0.116,0.126]
>>>>>>>> 效果不顯著 <<<<<<<<
